In [271]:
# !pip install apyori

In [272]:
import pandas as pd
import numpy as np
import numpy_indexed as npi
from itertools import combinations, product
from functools import reduce
from collections.abc import Iterable
import math
# use for vertify
from mlxtend.frequent_patterns import apriori, association_rules


In [273]:
testing_dataset_path = '/home/sokhorn/sokhorn/dataSet/data/sample_data_set.csv'
sample_dataset = pd.read_csv(
    testing_dataset_path, sep=',', usecols=[
        'InvoiceNo',
        'StockCode',
        'Quantity'
    ])
sample_dataset
item_sets = (
    sample_dataset.groupby(['InvoiceNo', 'StockCode', ])['Quantity']
    .sum().unstack().reset_index().fillna(0)
    .set_index("InvoiceNo")
)
# item_sets = item_sets.iloc[:, :10]
item_sets = item_sets.head(1000)
item_sets = item_sets.applymap(lambda x: 1 if x > 0 else 0)
item_sets.reindex(sorted(item_sets.columns), axis=1)
item_sets

StockCode,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
InvoiceNo,,,,,,,,,,,,,
536365,1,0,0,0,0,0,1,1,1,1,1,0,1
536366,0,1,1,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,1,1,1,0,0,0,0,0,1,0
536368,0,0,0,1,1,1,0,0,0,0,0,1,0
536369,0,0,0,1,1,1,0,0,0,0,0,1,0
536370,0,0,0,0,0,1,0,0,0,0,0,0,0
536371,0,0,0,0,0,1,0,0,0,0,0,0,0


#### Calculating RC count for each row of our data set 

In [274]:
# RC count generation 
itemset_reduct, rc_values =  npi.count(item_sets.values)

In [275]:
def rc(item_np):
    np_hash = {}
    item_rc = []
    for item in item_np:
        key = " ".join(map(str, item))
        if np_hash.get(key):
            np_hash[key] += 1
        else:
            np_hash[key] = 1

    for item in np_hash:
        values = list(map(int,  item.split()))
        values.append(np_hash[item])
        item_rc.append(values)
    return item_rc


In [276]:
# RC counr and tranction after reduce
rc_np = np.array(rc(item_sets.values))
df_rc = pd.DataFrame(rc_np)
columns = list(item_sets.columns)
columns.append("RC")
df_rc.set_axis(columns, inplace=True, axis=1)
RC = df_rc['RC']
df_rc.drop(['RC'], axis=1, inplace=True)
df_rc


,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
0,1,0,0,0,0,0,1,1,1,1,1,0,1
1,0,1,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,1,1,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0


In [277]:
# Support Count 
# reduce(function, input)
def support_k_itemst(k_itemst):
    s = 0
    for i in range(len(k_itemst)):
        s += reduce(lambda a, b: a & b, k_itemst[i] & RC[i])
    return s


**Generating 1 itemse base on support count of row**

In [278]:
C = {}
L = {}
k_items = []
Discard = {}
itemset_size = 1
min_support = 1
Discard.update({itemset_size: []})

### Remove every column which it support count are less than user defind support 

In [279]:
# remove each column who support count are less than min_support for 1 itemsets
rc_1_itemset = df_rc.sum(axis=0)
# remove all column which sum of row are ness than user defind support threshold
cut_our_cols = rc_1_itemset.loc[lambda x: x < min_support].index
# cut our every itemset which are sum of each row are less than user defind support
df_rc.drop(labels=cut_our_cols, axis=1, inplace=True)
# updpate the first item set which are in frequent 
Discard.update({itemset_size: cut_our_cols})

In [280]:
Discard

{1: Index([], dtype='object')}

In [281]:
# Frequent 1 itemsets
C.update({itemset_size: np.reshape(list(df_rc.columns), (-1, 1))})

In [282]:
def candidate_generation(Lk):
    try:
        k = len(Lk)
        k_1_item = len(Lk[0]) - 1
        for i in range(k):
            L1 = list(Lk[i])[-k_1_item::]
            for j in range(i + 1, k):
                L2 = list(Lk[j])[:k_1_item:]
                if L1 == L2:
                    yield list(np.union1d(Lk[i], Lk[j]))
                    # test next item the same or not, if no break this loop
                    if(j + 1 < k):
                        if((L1 != list(Lk[j + 1])[:k_1_item:])):
                            break

    except IndexError as err:
        print(f"Error with {err}")


In [283]:
# def get_frequent(itemesets, min_support, prev_discard):
#     L = []
#     support_count = []
#     new_discard = []
#     k = len(prev_discard)
#     for i in range(len(itemesets)):
#         discard_before = False
#         result = itemesets[i]

#         # if result in prev_discard than break this loop
#         if k > 0:
#             for it in prev_discard[k]:
#                 if set(it).issubset(set(result)):
#                     discard_before = True
#                     break
        
#         if not discard_before:
#             # print(f'item {mergeKItemIntoOne(item)}')
#             count = support_k_itemst(df_rc[result].values)
#             if count >= min_support:
#                 L.append(result)
#                 support_count.append(count)
#             else:
#                 new_discard.append(result)
#     return L, support_count, new_discard


In [284]:
def get_frequent(itemesets, min_support, prev_discard):
    k = len(prev_discard)
    for i in range(len(itemesets)):
        discard_before = False
        result = itemesets[i]

        # if result in prev_discard than break this loop
        if k > 0:
            for it in prev_discard[k]:
                if set(it).issubset(set(result)):
                    discard_before = True
                    break

        if not discard_before:
            # print(f'item {mergeKItemIntoOne(item)}')
            count = support_k_itemst(df_rc[result].values)
            status = True
            if count >= min_support:
                status = False
            else:
                status = True
            yield result, count, status

In [285]:
def filter_item(k_4):
    x_4 = np.stack(k_4,)
    support_count = x_4[:, 1]
    new_discard = x_4[np.where(x_4[:, 2] == True)][:, 0]
    L = x_4[np.where(x_4[:, 2] == False)][:, 0]
    return L, support_count, new_discard


In [286]:
def filter_candidate(can):
    return "Hello"

In [287]:
support_count = {}
f, supp, new_discard, = filter_item(
    get_frequent(C[itemset_size], min_support, Discard)
)
Discard.update({itemset_size: new_discard})
L.update({itemset_size: f})
support_count.update({itemset_size: supp})


/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3331: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/sokhorn/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [288]:
k = itemset_size + 1
k

2

In [289]:
c1 = np.array(C[1]).flatten()


def candidate_generation_1(Lk):
    try:
        k = len(Lk)
        for i in range(k):
            for j in range(i + 1, k):
                yield list(np.union1d(Lk[i], Lk[j]))
    except IndexError as err:
        print(f"Error with {err}")


C.update({k: list(candidate_generation_1(L[k - 1]))})
f, supp, new_discard = filter_item(get_frequent(C[k], min_support, Discard))
L.update({k: f})
Discard.update({k: new_discard})
support_count.update({k: supp})


In [290]:
k += 1

In [291]:
c_k = list(candidate_generation(L[k - 1]))
f, supp, new_discard = filter_item(
    get_frequent(c_k, min_support, Discard))


In [292]:

while True:
    try:
        C.update({k: np.stack(candidate_generation(L[k - 1]))})
        f, supp, new_discard = filter_item(
            get_frequent(C[k], min_support, Discard))
        L.update({k: f})
        Discard.update({k: new_discard})
        support_count.update({k: supp})
        if(len(L[k]) == 0):
            break
        k += 1
    except ValueError as err:
        print(f"Hello, there some error with {err}")
        break


Hello, there some error with need at least one array to stack


/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3254: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):
